In [1]:
cd ..

/Users/joeranbosma/stack/Projects/M5Forecast


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.environ['DATA_DIR'] = 'data/'
os.environ['SUB_DIR'] = 'submissions_uncertainty/'
for dirname, _, filenames in os.walk(os.environ['DATA_DIR']):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

data/.DS_Store
data/empty_pred.csv
data/pred_skeleton.csv
data/calendar.csv
data/sales_train_money.csv
data/sell_prices.csv
data/sales_train_validation.csv
data/sample_submission.csv


In [3]:
from flow import load_data

In [4]:
calendar, sales_train_val, sell_prices = load_data()

In [5]:
df = calendar[(calendar['year'] == 2011)].loc[:, ['wday', 'month', 'snap_CA']]
df.head()

,wday,month,snap_CA
date,,,
2011-01-29,1,1,0
2011-01-30,2,1,0
2011-01-31,3,1,0
2011-02-01,4,2,1
2011-02-02,5,2,1


In [6]:
df['mu'] = df.apply(lambda row: 1.5 if (row.wday==2 or row.wday==3) else 0.75, axis=1)

In [7]:
df['sigma'] = df.apply(lambda row: 0.25 if (row.wday==2 or row.wday==3) else 0.13, axis=1)

In [8]:
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]

In [9]:
targets = df.apply(lambda row: pd.Series(np.random.normal(row.mu, row.sigma, size=1000)).quantile(quantiles), axis=1)
targets.head()

,0.005,0.025,0.165,0.25,0.5,0.75,0.835,0.975,0.995
date,,,,,,,,,
2011-01-29,0.410956,0.494101,0.630373,0.671720,0.756835,0.843188,0.884852,1.006654,1.097859
2011-01-30,0.919469,0.992795,1.270255,1.332988,1.504236,1.675052,1.746716,2.014860,2.188142
2011-01-31,0.790407,0.988427,1.249410,1.322885,1.500305,1.659404,1.720131,1.953909,2.180639
2011-02-01,0.414008,0.487010,0.623712,0.659987,0.747542,0.837403,0.876519,1.000635,1.067878
2011-02-02,0.443139,0.511417,0.619042,0.662347,0.756661,0.838183,0.875509,0.997830,1.074211


In [10]:
df = pd.concat((df, pd.get_dummies(df.wday, prefix='w')), axis=1)
df.drop(columns=['mu', 'sigma'], inplace=True)
df.head()

,wday,month,snap_CA,w_1,w_2,w_3,w_4,w_5,w_6,w_7
date,,,,,,,,,,
2011-01-29,1,1,0,1,0,0,0,0,0,0
2011-01-30,2,1,0,0,1,0,0,0,0,0
2011-01-31,3,1,0,0,0,1,0,0,0,0
2011-02-01,4,2,1,0,0,0,1,0,0,0
2011-02-02,5,2,1,0,0,0,0,1,0,0


In [11]:
df.to_csv("custom_layer/features.csv")
targets.to_csv("custom_layer/targets.csv")